- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- 체인을 수동으로 구현해야 합니다.
- 체인에 ConversationBufferMemory를 부여합니다.
- 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- 체인에 다음 질문을 합니다:
Aaronson 은 유죄인가요?
그가 테이블에 어떤 메시지를 썼나요?
Julia 는 누구인가요?

다음과 같은 절차대로 구현하면 챌린지를 해결할 수 있습니다.
- (1) 문서 로드하기 : TextLoader 등 을 사용해서 파일에서 텍스트를 읽어옵니다. (Document Loaders 관련 문서
- (2) 문서 쪼개기 : CharacterTextSplitter 등 을 사용해서 문서를 작은 문서 조각들로 나눕니다. Character Split 관련 문서
- (3) 임베딩 생성 및 캐시 : OpenAIEmbeddings, CacheBackedEmbeddings 등 을 사용해 문서 조각들을 임베딩하고 임베딩을 저장합니다. Caching 관련 문서
- (4) 벡터 스토어 생성 : FAISS 등 을 사용해서 임베딩된 문서들을 저장하고 검색할 수 있는 데이터베이스를 만듭니다. FAISS 관련 문서
- (5) 대화 메모리와 질문 처리 : ConversationBufferMemory를 사용해 대화 기록을 관리합니다.
- (6) 체인 연결 : 앞에서 구현한 컴포넌트들을 적절하게 체인으로 연결합니다.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# 1. 문서 로드
loader = UnstructuredFileLoader("./document.txt")
documents = loader.load()

# 2. 문서 쪼개기
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = text_splitter.split_documents(documents)

# 3. 임베딩 설정
embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 4. 벡터 스토어 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 5. 메모리 설정
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# 6. 체인 생성
chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0),
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True,
)

In [4]:
# 질문 순차적으로 하기
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?", 
    "Julia 는 누구인가요?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    result = chain.invoke({"question": question})
    print(f"Answer: {result['answer']}")
    print("-" * 50)


Question: Aaronson 은 유죄인가요?
Answer: 저는 그 질문에 대한 답을 알 수 없습니다.
--------------------------------------------------

Question: 그가 테이블에 어떤 메시지를 썼나요?
Answer: 그가 쓴 메시지는 다음과 같습니다:
FREEDOM IS SLAVERY
TWO AND TWO MAKE FIVE
GOD IS POWER
--------------------------------------------------

Question: Julia 는 누구인가요?
Answer: Julia는 Winston과 함께 Party에 반항하고 사랑을 나누는 여성 캐릭터입니다. 그녀는 Winston에게 중요한 인물로 나타납니다.
--------------------------------------------------
